In [83]:
import pandas as pd

request = pd.read_excel('./sample_data.xlsx', sheet_name="request")
impression = pd.read_excel('./sample_data.xlsx', sheet_name="impression")
feedback = pd.read_excel('./sample_data.xlsx', sheet_name="feedback")

feedback.head(10)

,request_id,track_id,clicked,skipped_10s,listened_30s,liked,listened_sec
0,REQ_0001,TRK_00051,1,0,1,1,30
1,REQ_0001,TRK_00054,1,0,1,0,31
2,REQ_0001,TRK_00057,1,0,0,0,10
3,REQ_0001,TRK_00060,0,0,0,0,0
4,REQ_0001,TRK_00063,0,0,0,0,0
5,REQ_0002,TRK_00006,1,0,1,1,32
6,REQ_0002,TRK_00007,1,0,1,0,33
7,REQ_0002,TRK_00008,1,0,0,0,11
8,REQ_0002,TRK_00009,0,0,0,0,0
9,REQ_0002,TRK_00010,0,0,0,0,0


In [10]:
def validate_feedback(df):
    violations = []
    
    # 1) listened_sec ==0 
    mask = df["listened_sec"] == 0
    v = df[mask & (
        (df["clicked"] != 0) |
        (df["skipped_10s"] != 0) |
        (df["listened_30s"] != 0)
    )]
    if not v.empty:
        violations.append(("listened_sec == 0 rule", v))
        
    # 2) 1 ~ 9
    mask = df["listened_sec"].between(1, 9)
    v = df[mask & (
        (df["clicked"] != 1) |
        (df["skipped_10s"] != 1) |
        (df["listened_30s"] != 0)
    )]
    if not v.empty:
        violations.append(("1 <= listened_sec <= 9 rule", v))

    # 3) 10 ~ 29
    mask = df["listened_sec"].between(10, 29)
    v = df[mask & (
        (df["clicked"] != 1) |
        (df["skipped_10s"] != 0) |
        (df["listened_30s"] != 0)
    )]
    if not v.empty:
        violations.append(("10 <= listened_sec <= 29 rule", v))

    # 4) >= 30
    mask = df["listened_sec"] >= 30
    v = df[mask & (
        (df["clicked"] != 1) |
        (df["skipped_10s"] != 0) |
        (df["listened_30s"] != 1)
    )]
    if not v.empty:
        violations.append(("listened_sec >= 30 rule", v))

    # ---- 결과 출력 ----
    if not violations:
        print("✅ ALL CHECKS PASSED")
    else:
        print("❌ VIOLATIONS FOUND\n")
        for name, rows in violations:
            print(f"[{name}] → {len(rows)} rows")
            print(rows[[
                "request_id",
                "track_id",
                "clicked",
                "skipped_10s",
                "listened_30s",
                "listened_sec"
            ]].head(), "\n")


validate_feedback(feedback)

✅ ALL CHECKS PASSED


In [11]:
# impression <-> feedback 1:1대응 체크
assert (
    impression[["request_id", "track_id"]]
    .merge(feedback[["request_id", "track_id"]],
           on=["request_id", "track_id"],
           how="outer")
    .isnull()
    .sum()
    .sum() == 0
)

In [12]:
# 학습 단위(track 하나) 테이블 만들기
df = (
    impression
    .merge(feedback, on=["request_id", "track_id"], how="left")
    .merge(request, on="request_id", how="left")
)

df.head(10)

,policy_id,request_id,track_id,artist_id,rank,clicked,skipped_10s,listened_30s,liked,listened_sec,user_id,created_at,context_text,mood_vector,dominant_mood,mood_level_max,mood_entropy,situation_tag,place_tag,weather_tag
0,POL_RULE_V1,REQ_0001,TRK_00051,ART_0017,1,1,0,1,1,30,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
1,POL_RULE_V1,REQ_0001,TRK_00054,ART_0018,2,1,0,1,0,31,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
2,POL_RULE_V1,REQ_0001,TRK_00057,ART_0019,3,1,0,0,0,10,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
3,POL_RULE_V1,REQ_0001,TRK_00060,ART_0020,4,0,0,0,0,0,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
4,POL_RULE_V1,REQ_0001,TRK_00063,ART_0021,5,0,0,0,0,0,U_001,2025-01-01 08:12:00,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.42,commuting,home,cloudy
5,POL_RULE_V2,REQ_0002,TRK_00006,ART_0002,1,1,0,1,1,32,U_002,2025-01-01 22:45:00,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.31,resting,home,night
6,POL_RULE_V2,REQ_0002,TRK_00007,ART_0003,2,1,0,1,0,33,U_002,2025-01-01 22:45:00,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.31,resting,home,night
7,POL_RULE_V2,REQ_0002,TRK_00008,ART_0003,3,1,0,0,0,11,U_002,2025-01-01 22:45:00,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.31,resting,home,night
8,POL_RULE_V2,REQ_0002,TRK_00009,ART_0003,4,0,0,0,0,0,U_002,2025-01-01 22:45:00,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.31,resting,home,night
9,POL_RULE_V2,REQ_0002,TRK_00010,ART_0004,5,0,0,0,0,0,U_002,2025-01-01 22:45:00,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.31,resting,home,night


In [22]:
df.groupby("rank")["clicked"].mean()

rank
1    1.0
2    1.0
3    1.0
4    0.5
5    0.0
Name: clicked, dtype: float64

In [28]:
df["reward_score"] = (
    3 * df["liked"] +
    2 * df["listened_30s"] + 
    1 * df["skipped_10s"]
)

df.groupby("rank")["reward_score"].mean()

rank
1    3.48
2    1.16
3    0.80
4    1.48
5    0.12
Name: reward_score, dtype: float64

In [29]:
import numpy as np
import ast

#Shannon Entropy
def mood_entropy(mood_vector):
    x = np.array(mood_vector, dtype=float)
    if x.sum() == 0:
        return 0.0
    p = x / x.sum()
    p = p[p>0]
    return -np.sum(p*np.log(p))

df["mood_vector_parsed"] = df["mood_vector"].apply(ast.literal_eval) 
df["mood_entropy"] = df["mood_vector_parsed"].apply(mood_entropy)

df["mood_entropy"]

0      1.689245
1      1.689245
2      1.689245
3      1.689245
4      1.689245
         ...   
245    1.351784
246    1.351784
247    1.351784
248    1.351784
249    1.351784
Name: mood_entropy, Length: 250, dtype: float64

In [30]:
df.to_excel("./sample_data_2.xlsx")

In [31]:
df.columns

Index(['policy_id', 'request_id', 'track_id', 'artist_id', 'rank', 'clicked',
       'skipped_10s', 'listened_30s', 'liked', 'listened_sec', 'user_id',
       'created_at', 'context_text', 'mood_vector', 'dominant_mood',
       'mood_level_max', 'mood_entropy', 'situation_tag', 'place_tag',
       'weather_tag', 'reward_score', 'mood_vector_parsed'],
      dtype='object')

In [24]:
mood_cols = ["anger","sadness","pain","anxiety","shame","joy","love"]

mood_df = pd.DataFrame(df["mood_vector_parsed"].tolist(),
                       columns=[f"mood_{c}" for c in mood_cols])

mood_df

,mood_anger,mood_sadness,mood_pain,mood_anxiety,mood_shame,mood_joy,mood_love
0,1,7,3,4,1,2,1
1,1,7,3,4,1,2,1
2,1,7,3,4,1,2,1
3,1,7,3,4,1,2,1
4,1,7,3,4,1,2,1
...,...,...,...,...,...,...,...
245,0,2,0,1,0,2,2
246,0,2,0,1,0,2,2
247,0,2,0,1,0,2,2
248,0,2,0,1,0,2,2


In [25]:
dominant_mood = pd.get_dummies(df["dominant_mood"], 
                               columns=["dominant_mood"], 
                               prefix="dominant_mood")

situation_tag = pd.get_dummies(df["situation_tag"],
                               columns=["situation_tag"],
                               prefix="situation")

place_tag = pd.get_dummies(df["place_tag"],
                           columns=["place_tag"],
                           prefix="place")

weather_tag = pd.get_dummies(df["weather_tag"],
                             columns=["weather_tag"],
                             prefix="weather")

weather_tag

,weather_clear,weather_cloudy,weather_night,weather_rain
0,False,True,False,False
1,False,True,False,False
2,False,True,False,False
3,False,True,False,False
4,False,True,False,False
...,...,...,...,...
245,False,False,True,False
246,False,False,True,False
247,False,False,True,False
248,False,False,True,False


In [84]:
X = pd.concat([mood_df, dominant_mood, df["mood_level_max"], df["mood_entropy"],
                situation_tag, place_tag, weather_tag,
                df["rank"]], axis=1)

X

,mood_anger,mood_sadness,mood_pain,mood_anxiety,mood_shame,mood_joy,mood_love,dominant_mood_anxiety,dominant_mood_joy,dominant_mood_love,...,place_gym,place_home,place_office,place_outdoor,place_transport,weather_clear,weather_cloudy,weather_night,weather_rain,rank
0,1,7,3,4,1,2,1,False,False,False,...,False,True,False,False,False,False,True,False,False,1
1,1,7,3,4,1,2,1,False,False,False,...,False,True,False,False,False,False,True,False,False,2
2,1,7,3,4,1,2,1,False,False,False,...,False,True,False,False,False,False,True,False,False,3
3,1,7,3,4,1,2,1,False,False,False,...,False,True,False,False,False,False,True,False,False,4
4,1,7,3,4,1,2,1,False,False,False,...,False,True,False,False,False,False,True,False,False,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0,2,0,1,0,2,2,False,True,False,...,False,True,False,False,False,False,False,True,False,1
246,0,2,0,1,0,2,2,False,True,False,...,False,True,False,False,False,False,False,True,False,2
247,0,2,0,1,0,2,2,False,True,False,...,False,True,False,False,False,False,False,True,False,3
248,0,2,0,1,0,2,2,False,True,False,...,False,True,False,False,False,False,False,True,False,4


In [85]:
y = df["reward_score"].astype(float)

y

0      5.0
1      2.0
2      0.0
3      0.0
4      0.0
      ... 
245    2.0
246    0.0
247    0.0
248    0.0
249    0.0
Name: reward_score, Length: 250, dtype: float64

In [86]:
sort_cols = ["request_id", "rank"]

df = df.sort_values(sort_cols).reset_index(drop=True)
X = X.loc[df.index].reset_index(drop=True).astype(float)
y = y.loc[df.index].reset_index(drop=True).astype(float)

X

,mood_anger,mood_sadness,mood_pain,mood_anxiety,mood_shame,mood_joy,mood_love,dominant_mood_anxiety,dominant_mood_joy,dominant_mood_love,...,place_gym,place_home,place_office,place_outdoor,place_transport,weather_clear,weather_cloudy,weather_night,weather_rain,rank
0,1.0,7.0,3.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,7.0,3.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
2,1.0,7.0,3.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0
3,1.0,7.0,3.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0
4,1.0,7.0,3.0,4.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.0,2.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
246,0.0,2.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
247,0.0,2.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0
248,0.0,2.0,0.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0


In [87]:
req_ids = df["request_id"].unique()
rng = np.random.default_rng(42) 
rng.shuffle(req_ids)

In [88]:
n_train = int(len(req_ids)*0.8) 
train_reqs = set(req_ids[:n_train])
test_reqs = set(req_ids[n_train:])

train_mask = df["request_id"].isin(train_reqs)
test_mask = df["request_id"].isin(test_reqs)

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

df_test = df[test_mask].copy()

df_test

,policy_id,request_id,track_id,artist_id,rank,clicked,skipped_10s,listened_30s,liked,listened_sec,...,context_text,mood_vector,dominant_mood,mood_level_max,mood_entropy,situation_tag,place_tag,weather_tag,reward_score,mood_vector_parsed
0,POL_RULE_V1,REQ_0001,TRK_00051,ART_0017,1,1,0,1,1,30,...,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.689245,commuting,home,cloudy,5,"[1, 7, 3, 4, 1, 2, 1]"
1,POL_RULE_V1,REQ_0001,TRK_00054,ART_0018,2,1,0,1,0,31,...,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.689245,commuting,home,cloudy,2,"[1, 7, 3, 4, 1, 2, 1]"
2,POL_RULE_V1,REQ_0001,TRK_00057,ART_0019,3,1,0,0,0,10,...,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.689245,commuting,home,cloudy,0,"[1, 7, 3, 4, 1, 2, 1]"
3,POL_RULE_V1,REQ_0001,TRK_00060,ART_0020,4,0,0,0,0,0,...,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.689245,commuting,home,cloudy,0,"[1, 7, 3, 4, 1, 2, 1]"
4,POL_RULE_V1,REQ_0001,TRK_00063,ART_0021,5,0,0,0,0,0,...,출근 준비하면서 기분이 좀 가라앉아,"[1,7,3,4,1,2,1]",sadness,7,1.689245,commuting,home,cloudy,0,"[1, 7, 3, 4, 1, 2, 1]"
5,POL_RULE_V2,REQ_0002,TRK_00006,ART_0002,1,1,0,1,1,32,...,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.586785,resting,home,night,5,"[0, 6, 2, 3, 1, 1, 2]"
6,POL_RULE_V2,REQ_0002,TRK_00007,ART_0003,2,1,0,1,0,33,...,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.586785,resting,home,night,2,"[0, 6, 2, 3, 1, 1, 2]"
7,POL_RULE_V2,REQ_0002,TRK_00008,ART_0003,3,1,0,0,0,11,...,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.586785,resting,home,night,0,"[0, 6, 2, 3, 1, 1, 2]"
8,POL_RULE_V2,REQ_0002,TRK_00009,ART_0003,4,0,0,0,0,0,...,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.586785,resting,home,night,0,"[0, 6, 2, 3, 1, 1, 2]"
9,POL_RULE_V2,REQ_0002,TRK_00010,ART_0004,5,0,0,0,0,0,...,혼자 방에서 누워서 멍 때리고 있어,"[0,6,2,3,1,1,2]",sadness,6,1.586785,resting,home,night,0,"[0, 6, 2, 3, 1, 1, 2]"


In [89]:
group_train = df.loc[train_mask].groupby("request_id").size().to_list()
group_test = df.loc[test_mask].groupby("request_id").size().to_list()

In [56]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [93]:
from xgboost import XGBRanker

model = XGBRanker(
    objective="rank:ndcg",
    eval_metric="ndcg",
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
)

model.fit(
    X_train, y_train,
    group=group_train,
    eval_set=[(X_test, y_test)],
    eval_group=[group_test],
    verbose=False
)

,objective,'rank:ndcg'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'ndcg'


In [94]:
df_test["pred_score"] = model.predict(X_test)

topk = (
    df_test.sort_values(["request_id", "pred_score"], ascending=[True, False])
            .groupby("request_id")
            .head(5)[["request_id", "track_id", "pred_score", "reward_score", "rank"]]
)

topk

,request_id,track_id,pred_score,reward_score,rank
0,REQ_0001,TRK_00051,1.823180,5,1
1,REQ_0001,TRK_00054,-0.505431,2,2
2,REQ_0001,TRK_00057,-0.777319,0,3
3,REQ_0001,TRK_00060,-1.015582,0,4
4,REQ_0001,TRK_00063,-2.142179,0,5
5,REQ_0002,TRK_00006,0.117751,5,1
8,REQ_0002,TRK_00009,-0.031013,0,4
7,REQ_0002,TRK_00008,-0.109976,0,3
6,REQ_0002,TRK_00007,-0.200250,2,2
9,REQ_0002,TRK_00010,-1.889664,0,5


In [114]:
# 모델이 추천한 상위 K곡을 사용자에게 보여줬을 때, 요청 하나당 얻는 총 만족도의 평균

def mean_reward(df, score_col="pred_score", k=3):
    return (
        df.sort_values(["request_id", score_col], ascending=[True, False])
            .groupby("request_id")
            .head(k)
            .groupby("request_id")["reward_score"]
            .sum()
            .mean()
    )

for k in range(1,6):
    model_score = mean_reward(df_test, "pred_score", k)   
    baseline_score = mean_reward(df_test, "rank", k)  
    print(model_score, baseline_score)



3.9 0.0
5.3 0.8
6.7 1.4
6.9 2.8
6.9 6.9


5위까지 순위를 가진 데이터로 모델을 학습시킨 결과
- 5위까지 순위 예측: 모델 성능 6.9, 원래 성능 6.9
- 4위까지 순위 예측: 모델 성능 6.9, 원래 성능 2.8 
- 3위까지 순위 예측: 모델 성능 6.7, 원래 성능 1.4 (점점 떨어짐)
- 2위까지 순위 예측: 모델 성능 5.3, 원래 성능 0.8
- 1위까지 순위 예측: 모델 성능 3.9, 원래 성능 0.0 (예측할 수 없음?)

원래 데이터까지고는 3,4위까지 예측하는 것은 무리가 있음
이런 경우엔 모델이 기존 노출 순서보다 더 나은 추천을 함

In [118]:
def metric(df, col, score_col="pred_score", k=3):
    return (
        df.sort_values(["request_id", score_col], ascending=[True, False])
            .groupby("request_id")
            .head(k)[col]
            .mean()
    )
    
for k in range(1,6):
    print("K=", k,
          "reward@", mean_reward(df_test, "pred_score", k),
          "click@", metric(df_test, "clicked", "pred_score", k),
          "like@", metric(df_test, "liked", "pred_score", k)
    )

K= 1 reward@ 3.9 click@ 0.9 like@ 0.7
K= 2 reward@ 5.3 click@ 0.85 like@ 0.4
K= 3 reward@ 6.7 click@ 0.8333333333333334 like@ 0.3333333333333333
K= 4 reward@ 6.9 click@ 0.825 like@ 0.25
K= 5 reward@ 6.9 click@ 0.66 like@ 0.2
